In [50]:
import requests

In [51]:
import geopandas as gpd
import pandas as pd

## Open State data

In [52]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Asian Data

In [53]:
table = "B02018"

url = "https://api.census.gov/data/2023/acs/acs5"
params = {
    "get": f"group({table})",
    "ucgid": "pseudo(0100000US$0400000)",
}
response = requests.get(url, params=params)

In [54]:
data = response.json()
columns = data[0]
rows = data[1:]
asian_df = pd.DataFrame(rows, columns=columns)

In [55]:
url = "https://api.census.gov/data/2023/acs/acs5/variables.json"
response = requests.get(url)
variables = response.json()["variables"]

In [57]:
b02018_vars = {k: v["label"] for k, v in variables.items() if k.startswith(table)}
b02018_vars["ucgid"] = "AFFGEOID"

In [58]:
asian_df = asian_df.rename(columns=b02018_vars)
asian_df = asian_df[list(b02018_vars.values())]

In [59]:
rename_dict = {
    col: col.split("!!")[-1]
    for col in asian_df.columns
    if col.startswith("Estimate!!Total")
}
asian_df.rename(columns=rename_dict, inplace=True)

In [60]:
ethnicity_cols = list(rename_dict.values())
asian_df[ethnicity_cols] = asian_df[ethnicity_cols].astype(int)

In [61]:
asian_df['STATEFP'] = asian_df['AFFGEOID'].str[9:]
japanese_df = asian_df[['STATEFP', 'Japanese']]

## Get Population data

In [62]:
url = "https://api.census.gov/data/2023/acs/acs5/subject"
params = params = {
    "get": "NAME,S0101_C01_001E",
    "for": "state:*"
}
response = requests.get(url, params=params)

In [63]:
data = response.json()
columns = data[0]
rows = data[1:]
population_df = pd.DataFrame(rows, columns=columns)

In [64]:
population_df = population_df.rename(columns={"S0101_C01_001E": "POPULATION", "state":"STATEFP"})

## Merge Data

In [65]:
japanese_population_df = population_df.merge(japanese_df, how='inner', on='STATEFP')

In [66]:
japanese_population_df[['Japanese', 'POPULATION']] = japanese_population_df[['Japanese', 'POPULATION']].astype(int)

In [67]:
japanese_population_df["per_100k"] = (
    (japanese_population_df["Japanese"] / (japanese_population_df["POPULATION"] / 100_000)).round(decimals=0).astype(int)
)

In [68]:
japanese_population_df["PERCENT"] = (
    (
        japanese_population_df["Japanese"] / japanese_population_df["POPULATION"]
    ) * 100.0
).round(decimals=5).astype(float)

In [69]:
japanese_population_gdf = states_df.merge(japanese_population_df, on="STATEFP", how="inner")

In [70]:
japanese_population_gdf = japanese_population_gdf[['PERCENT', 'geometry', 'per_100k']]

In [71]:
japanese_population_gdf = japanese_population_gdf.to_crs(9311)
japanese_population_gdf.to_file("data/japanese_per_state.gpkg")